# Important Pandas Functions

Adapted from Wes McKinney's Python for Data Analysis and the Pandas Documentation

In [1]:
import numpy as np
import pandas as pd

# Reindexing

Reindexing takes a current Pandas object and creates a *new* Pandas object that *conforms* to the specified index:

In [2]:
original = pd.Series([1.4, 2.3, 3.1, 4.2], index = ['d','c','a','b'])

In [3]:
original

d    1.4
c    2.3
a    3.1
b    4.2
dtype: float64

In [4]:
newobj = original.reindex(['a','b','c','d','e'])

In [5]:
newobj  # takes the data in orignal and moves it so it conforms to the specified index
# values that do not exist for the new index get NaN

a    3.1
b    4.2
c    2.3
d    1.4
e    NaN
dtype: float64

In [6]:
# if you don't want NaN, you can specify a fill_value
newobj2 = original.reindex(['a','b','c','d','e'], fill_value = 0)
newobj2

a    3.1
b    4.2
c    2.3
d    1.4
e    0.0
dtype: float64

For ordered data like a time series, it might be desirable to fill values when reindexing

In [7]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 3, 6])
obj3

0      blue
3    purple
6    yellow
dtype: object

In [8]:
obj3.reindex(range(9))  # without any optional arguments, lots of missing values

0      blue
1       NaN
2       NaN
3    purple
4       NaN
5       NaN
6    yellow
7       NaN
8       NaN
dtype: object

In [9]:
obj3.reindex(range(9), method='ffill')
# forward-fill pushes values 'forward' until a new value is encountered

0      blue
1      blue
2      blue
3    purple
4    purple
5    purple
6    yellow
7    yellow
8    yellow
dtype: object

In [10]:
obj3.reindex(range(9), method='bfill')  
# back-fill works in the opposite direction
# there was no value at index 8 so, NaNs get filled in

0      blue
1    purple
2    purple
3    purple
4    yellow
5    yellow
6    yellow
7       NaN
8       NaN
dtype: object

In [11]:
# we specify the creation of a date_index using the date_range function
# freq = 'D' creates Daily values 
date_index = pd.date_range('1/1/2010', periods=6, freq='D')
date_index

DatetimeIndex(['2010-01-01', '2010-01-02', '2010-01-03', '2010-01-04',
               '2010-01-05', '2010-01-06'],
              dtype='datetime64[ns]', freq='D')

In [12]:
# we create a DataFrame with the date index
df2 = pd.DataFrame({"prices": [100, 101, np.nan, 100, 89, 88]}, index=date_index)
df2

,prices
2010-01-01,100.0
2010-01-02,101.0
2010-01-03,NaN
2010-01-04,100.0
2010-01-05,89.0
2010-01-06,88.0


In [13]:
date_index2 = pd.date_range('12/29/2009', periods=10, freq='D')  # a new date index
df2.reindex(date_index2)

,prices
2009-12-29,NaN
2009-12-30,NaN
2009-12-31,NaN
2010-01-01,100.0
2010-01-02,101.0
2010-01-03,NaN
2010-01-04,100.0
2010-01-05,89.0
2010-01-06,88.0
2010-01-07,NaN


In [14]:
df2.reindex(date_index2, method = 'bfill') # let me look up why the jan 3 isn't filled in

,prices
2009-12-29,100.0
2009-12-30,100.0
2009-12-31,100.0
2010-01-01,100.0
2010-01-02,101.0
2010-01-03,NaN
2010-01-04,100.0
2010-01-05,89.0
2010-01-06,88.0
2010-01-07,NaN


## `.reindex()` vs `.loc()`

If you don't need to fill in any missing info, then .reindex() and .loc() work the same
If the new index will have values that don't exist in the current index, you need to use reindex

In [15]:
obj5 = pd.DataFrame({'val':[1.4, 2.3, 3.1, 4.2]}, index = ['d','c','a','b'])
obj5

,val
d,1.4
c,2.3
a,3.1
b,4.2


In [16]:
obj5.reindex(['a','b','c','d'])

,val
a,3.1
b,4.2
c,2.3
d,1.4


In [17]:
obj5.loc[['a','b','c','d']] # works the same as reindex

,val
a,3.1
b,4.2
c,2.3
d,1.4


In [18]:
obj5.reindex(['a','b','c','d','e'])

,val
a,3.1
b,4.2
c,2.3
d,1.4
e,NaN


In [19]:
obj5.loc[['a','b','c','d','e']]  # .loc() returns a warning or error if you give an entry in the index that doesn't exist

C:\Users\miles\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,val
a,3.1
b,4.2
c,2.3
d,1.4
e,NaN


# Dropping rows or columns

you can use `df.drop()` to remove rows (default) or columns (specify axis = 1) at certain index locations.

In [20]:
df = pd.DataFrame(np.arange(12).reshape(3,4), columns=['A', 'B', 'C', 'D'], index = ['a','b','c'])
df

,A,B,C,D
a,0,1,2,3
b,4,5,6,7
c,8,9,10,11


In [21]:
# drop rows
df.drop(['a', 'c'])

,A,B,C,D
b,4,5,6,7


In [22]:
# drop columns
df.drop(['B', 'C'], axis=1)

,A,D
a,0,3
b,4,7
c,8,11


In [23]:
# df.drop returns a new object and leaves df unchanged
# you can change this behavior with the argument inplace = True
df

,A,B,C,D
a,0,1,2,3
b,4,5,6,7
c,8,9,10,11


# Data Alignment

When performing element-wise arithmetic, Pandas will align the index values before doing the computation

In [24]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [25]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=['a', 'c', 'e', 'f', 'g'])
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [26]:
s1 + s2  # returns a new series, where the indexes are the union of the indexes of s1 and s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

For data frames with different columns, the rows and columns will be aligned

In [27]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [28]:
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [29]:
df1 + df2 
# c is in df1, but not df2
# e is in df2, but not df1
# the result returns the union of columns, but will fill in NaN for elements that do not exist in both

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


In [30]:
# if you want to fill in values that are missing, you can use df.add() and specify the fill_value
# this will perform the above operation, but instead of using NaN when it can't find a value 
# (which will return NaN),
# it will use the fill_value
df1.add(df2, fill_value = 0)
# you still get NaN if the value does not exist in either DataFrame

,b,c,d,e
Colorado,6.0,7.0,8.0,NaN
Ohio,3.0,1.0,6.0,5.0
Oregon,9.0,NaN,10.0,11.0
Texas,9.0,4.0,12.0,8.0
Utah,0.0,NaN,1.0,2.0


In [31]:
# other arithmetic operations that can be called on DataFrames are:
# .add()
# .sub()
# .mul()
# .div()
# .floordiv()

# Summary Stats of a DataFrame

In [32]:
df = pd.DataFrame({'one':[1.5,6.0,np.nan, 1.5,4,6, np.nan],
                   'two':[np.nan, -4.5, np.nan, -1.5, 0, -4.5, 4]},
                  index=['a', 'b', 'c', 'd','e','f','g'])
df

,one,two
a,1.5,NaN
b,6.0,-4.5
c,NaN,NaN
d,1.5,-1.5
e,4.0,0.0
f,6.0,-4.5
g,NaN,4.0


In [33]:
df.sum()  # default behavior returns column sums and skips missing values
# default behavior sums across axis 0 (sums the row)

one    19.0
two    -6.5
dtype: float64

In [34]:
df.sum(axis = 1) # sum across axis=1, sum across the columns and give row sums

a    1.5
b    1.5
c    0.0
d    0.0
e    4.0
f    1.5
g    4.0
dtype: float64

In [35]:
df.sum(skipna = False)

one   NaN
two   NaN
dtype: float64

In [36]:
df.mean()

one    3.8
two   -1.3
dtype: float64

In [37]:
df.mean(axis = 1)

a    1.50
b    0.75
c     NaN
d    0.00
e    2.00
f    0.75
g    4.00
dtype: float64

In [38]:
df.idxmin()  # which row has the minimum value, also .idxmax()
# returns the first minimum, if there are multiple

one    a
two    b
dtype: object

In [39]:
df.one.unique()  # shows the unique values in the order observed

array([1.5, 6. , nan, 4. ])

In [40]:
df.one.nunique()  # number of non-missing unique values exist

3

In [41]:
df.one.value_counts()  # tally up counts of each value
# returns a series. the index are the unique values observed, the values are the frequencies.
# they appear in descending order of frequency

6.0    2
1.5    2
4.0    1
Name: one, dtype: int64

In [42]:
df.one.isin([1.5, 4.0]) # checks to see if the value has membership in a particular list
# returns a series with boolean values

a     True
b    False
c    False
d     True
e     True
f    False
g    False
Name: one, dtype: bool

In [43]:
df.loc[  df.one.isin([1.5,4.0]),  ]  # can filter rows based on the .isin() membership

,one,two
a,1.5,NaN
d,1.5,-1.5
e,4.0,0.0


# filtering out missing values

In [44]:
df

,one,two
a,1.5,NaN
b,6.0,-4.5
c,NaN,NaN
d,1.5,-1.5
e,4.0,0.0
f,6.0,-4.5
g,NaN,4.0


In [45]:
df.dropna() # gets rid of any row that is not complete

,one,two
b,6.0,-4.5
d,1.5,-1.5
e,4.0,0.0
f,6.0,-4.5


In [46]:
df.dropna(how = 'all')  # only drops rows that are entirely NaN

,one,two
a,1.5,NaN
b,6.0,-4.5
d,1.5,-1.5
e,4.0,0.0
f,6.0,-4.5
g,NaN,4.0


In [47]:
# you can also use .notnull(), which is True for values that are not missing
df[df.two.notnull()]  # You can use this in conjuntion with specifying the column

,one,two
b,6.0,-4.5
d,1.5,-1.5
e,4.0,0.0
f,6.0,-4.5
g,NaN,4.0


# Filling in Missing Values

In [48]:
df

,one,two
a,1.5,NaN
b,6.0,-4.5
c,NaN,NaN
d,1.5,-1.5
e,4.0,0.0
f,6.0,-4.5
g,NaN,4.0


In [49]:
df.fillna(0) # fill in missing values with a constant

,one,two
a,1.5,0.0
b,6.0,-4.5
c,0.0,0.0
d,1.5,-1.5
e,4.0,0.0
f,6.0,-4.5
g,0.0,4.0


In [50]:
df.fillna({'one': 1000, 'two': 0})  # use a dictionary to specify values to use for each column

,one,two
a,1.5,0.0
b,6.0,-4.5
c,1000.0,0.0
d,1.5,-1.5
e,4.0,0.0
f,6.0,-4.5
g,1000.0,4.0


In [51]:
df.fillna(method = 'bfill')  # backfills. You can also use ffill

,one,two
a,1.5,-4.5
b,6.0,-4.5
c,1.5,-1.5
d,1.5,-1.5
e,4.0,0.0
f,6.0,-4.5
g,NaN,4.0


In [52]:
df.mean(axis = 1)

a    1.50
b    0.75
c     NaN
d    0.00
e    2.00
f    0.75
g    4.00
dtype: float64

In [53]:
df.fillna(df.mean())  # fill na with df.mean() will fill in the column means

,one,two
a,1.5,-1.3
b,6.0,-4.5
c,3.8,-1.3
d,1.5,-1.5
e,4.0,0.0
f,6.0,-4.5
g,3.8,4.0


all of the above fillna methods have created new DataFrame objects. If you want to modify the current DataFrame, you can use the optional argument `inplace = True`

In [54]:
df.T

,a,b,c,d,e,f,g
one,1.5,6.0,NaN,1.5,4.0,6.0,NaN
two,NaN,-4.5,NaN,-1.5,0.0,-4.5,4.0


In [55]:
# apparently you can only fill missing values with dictionaries/series over a column 
# so we have to do some Transpose magic
df.T.fillna(df.T.mean()).T

,one,two
a,1.5,1.5
b,6.0,-4.5
c,NaN,NaN
d,1.5,-1.5
e,4.0,0.0
f,6.0,-4.5
g,4.0,4.0


# dealing with duplicates

In [56]:
df

,one,two
a,1.5,NaN
b,6.0,-4.5
c,NaN,NaN
d,1.5,-1.5
e,4.0,0.0
f,6.0,-4.5
g,NaN,4.0


In [57]:
df.duplicated()  # sees if any of the rows are a duplicate of an earlier row

a    False
b    False
c    False
d    False
e    False
f     True
g    False
dtype: bool

In [58]:
df[~df.duplicated()]  # gets rid of the duplicated rows

,one,two
a,1.5,NaN
b,6.0,-4.5
c,NaN,NaN
d,1.5,-1.5
e,4.0,0.0
g,NaN,4.0


In [59]:
df.one.duplicated()

a    False
b    False
c    False
d     True
e    False
f     True
g     True
Name: one, dtype: bool